In [1]:
import pandas as pd
import requests
import numpy
import sqlite3
import os
import time

In [2]:
# Get my building blocks
Item_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Item URLs.csv")
Resource_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Resource URLs.csv")
Artefact_Journal_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Artefact and Journal URLs.csv")
Recipies = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Albion_Equipment_Recipies_3.csv")

In [3]:
# this function lets us create a dataframe out of a list of URL's for Albion Data Project GET API.
def Get_Data2(URL_list):
    df_data = pd.DataFrame(columns=['item_id', 'city', 'quality', 'sell_price_min', 'sell_price_min_date', 'sell_price_max', 'sell_price_max_date', 'buy_price_min', 'buy_price_min_date', 'buy_price_max', 'buy_price_max_date'])
    for i in range(len(URL_list)):
        print(f'working on URL #{i}')
        Current_URL = URL_list['URL'][i]
        URL_Contents = requests.get(url=Current_URL).json()
        URL_Contents = pd.DataFrame(URL_Contents)
        df_data = pd.concat([df_data,URL_Contents])
    print('API calls done')
    return df_data

# a function to take old and new prices, and drop duplicates, with the duplicates kept being the data from most recent prices.
def Update_Prices(hitorical_prices, updated_prices):
    merged = pd.concat([hitorical_prices, updated_prices], ignore_index=True)
    merged.drop_duplicates(subset=['item_id', 'city', 'quality'], keep='last', inplace=True)
    return merged

In [4]:
def Get_Data2(URL_list):
    df_data = pd.DataFrame(columns=['item_id', 'city', 'quality', 'sell_price_min', 'sell_price_min_date', 'sell_price_max', 'sell_price_max_date', 'buy_price_min', 'buy_price_min_date', 'buy_price_max', 'buy_price_max_date'])
    for i in range(len(URL_list)):
        print(f'working on URL #{i}')
        Current_URL = URL_list['URL'][i]
        URL_Contents = requests.get(url=Current_URL).json()
        URL_Contents = pd.DataFrame(URL_Contents)
        df_data = pd.concat([df_data,URL_Contents])
    print('API calls done')
    return df_data

In [5]:
# Make a fresh call to API to get recent data.
resource_prices_update = Get_Data2(Resource_URLs)
artefact_and_journal_prices_update = Get_Data2(Artefact_Journal_URLs)
time.sleep(61) #There is a 180 calls per minute limit that we must avoid. also 300 per 5 minutes limit.
item_prices_update = Get_Data2(Item_URLs)


working on URL #0
working on URL #1
working on URL #2
API calls done
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
working on URL #16
working on URL #17
working on URL #18
working on URL #19
working on URL #20
working on URL #21
working on URL #22
working on URL #23
working on URL #24
working on URL #25
working on URL #26
working on URL #27
working on URL #28
working on URL #29
working on URL #30
working on URL #31
API calls done
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
working on URL #16
worki

In [6]:
#items that have no data are present but just with 0's in price data.
item_prices_update = item_prices_update[item_prices_update['sell_price_min']>0]
resource_prices_update = resource_prices_update[resource_prices_update['sell_price_min']>0]
artefact_and_journal_prices_update = artefact_and_journal_prices_update[artefact_and_journal_prices_update['sell_price_min']>0]


In [7]:
# Get prices used in last pull:
resource_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv")
artefact_and_journal_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv")
item_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv")


In [8]:
#lets update our item prices now
item_prices_final = Update_Prices(item_prices_historical, item_prices_update)
resource_prices_final = Update_Prices(resource_prices_historical, resource_prices_update)
artefact_and_journal_prices_final = Update_Prices(artefact_and_journal_prices_historical, artefact_and_journal_prices_update)


In [9]:
#and now we must save this new data into our csv files so they can be used the next time I run this program to update prices.

#top 3 are normal process, bottom 3 are to refresh the historical data files.

resource_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
artefact_and_journal_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
item_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

#resource_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
#artefact_and_journal_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
#item_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

In [10]:
# Now that we have updated our prices and saved them, we can start to manipulate the price data into our report, starting with trimming columns we dont need.

item_prices_report = item_prices_final
artefact_and_journal_prices_report = artefact_and_journal_prices_final
resource_prices_report = resource_prices_final

item_prices_report = item_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]
item_prices_report = item_prices_report.groupby(['item_id', 'city']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #items have qualities. I looked for all qualities to "widen the net" we cast with our API call. Time to group by item so that each only has 1 row.
item_prices_report['sell_price_min'] = item_prices_report['sell_price_min'].astype(int)

artefact_and_journal_prices_report = artefact_and_journal_prices_report.groupby(['item_id']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #these items NEVER get checked by players, and unlike items or resources, have no attachment to geography. for this reason I check all cities and group by just item to once again "widen the net"
artefact_and_journal_prices_report['sell_price_min'] = artefact_and_journal_prices_report['sell_price_min'].astype(int)

resource_prices_report = resource_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]


In [11]:
# Moving to SQL for all the merges we are about to do.
Albion_DB = "../Albion.sqlite"
Albion_conn = sqlite3.connect(Albion_DB)

item_prices_report.to_sql(name='item_prices_report', if_exists='replace', con=Albion_conn)
artefact_and_journal_prices_report.to_sql(name='artefact_and_journal_prices_report', if_exists='replace', con=Albion_conn)
resource_prices_report.to_sql(name='resource_prices_report', if_exists='replace', con=Albion_conn)
Recipies.to_sql(name='Recipies', if_exists='replace', con=Albion_conn)

4240

In [12]:
BM_Craft = pd.read_sql_query(
"""
with add_sell_price as (
SELECT 
Recipies.*

,item_prices_report.city as "Sell_City"
,item_prices_report.sell_price_min as "Sell_Price"
,item_prices_report.sell_price_min_date as "Sell_Price_Date"

FROM Recipies LEFT JOIN item_prices_report
ON Recipies.Item_Placeholder = item_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_prim_res as (
SELECT 
add_sell_price.*

,resource_prices_report.city as "Main_Resource_City"
,resource_prices_report.sell_price_min as 'Main_Resource_Price'
,resource_prices_report.sell_price_min_date as 'Main_Resource_Date'
 
FROM add_sell_price LEFT JOIN resource_prices_report
ON add_sell_price.Main_Resource = resource_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_sec_res as (
SELECT 
add_prim_res.*

,resource_prices_report.city as "Secondary_Resource_City"
,resource_prices_report.sell_price_min as 'Secondary_Resource_Price'
,resource_prices_report.sell_price_min_date as 'Secondary_Resource_Date'

FROM add_prim_res LEFT JOIN resource_prices_report
ON add_prim_res.Secondary_Resource = resource_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_artefact as (
SELECT 
add_sec_res.*

,artefact_and_journal_prices_report.sell_price_min as 'Artefact_Price'
,artefact_and_journal_prices_report.sell_price_min_date as 'Artefact_Date'

FROM add_sec_res LEFT JOIN artefact_and_journal_prices_report
ON add_sec_res.Artefact_Placeholder = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_journal_empty as (
SELECT 
add_artefact.*

,artefact_and_journal_prices_report.sell_price_min as 'Journal_Empty_Price'

FROM add_artefact LEFT JOIN artefact_and_journal_prices_report
ON add_artefact.Journal_Empty = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_journal_full as (
SELECT 
add_journal_empty.*

,artefact_and_journal_prices_report.sell_price_min as 'Journal_Full_Price'

,CASE WHEN Artefact_Placeholder<>0 AND Artefact_Price IS NULL THEN 1 ELSE 0 END AS 'Missing_Artefact'
,CASE WHEN Secondary_Resource_Quantity<>'0' AND Secondary_Resource_Price IS NULL THEN 1 ELSE 0 END AS 'Missing_Second_Resource'

FROM add_journal_empty LEFT JOIN artefact_and_journal_prices_report
ON add_journal_empty.Journal_Full = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------

SELECT
*

,CASE WHEN Secondary_Resource_Price IS NULL THEN 0 ELSE Secondary_Resource_Price END as 'Secondary_Resource_Price_nn'
,CASE WHEN Artefact_Price IS NULL THEN 0 ELSE Artefact_Price END as 'Artefact_Price_nn'
,Tier||'.'||Enchant as Tier_Enchant

FROM add_journal_full
WHERE Sell_Price IS NOT NULL AND Missing_Artefact=0 AND Missing_Second_Resource=0 AND Main_Resource_Price>0 AND Journal_Empty_Price IS NOT NULL AND Journal_Full_Price IS NOT NULL and Sell_City IN ('Black Market','Fort Sterling');




 """, Albion_conn)

In [13]:
BM_Craft.columns


Index(['index', 'Item_Placeholder', 'Tier', 'Enchant', 'Item_Name',
       'Main_Resource', 'Main Resource Quantity', 'Secondary_Resource',
       'Secondary_Resource_Quantity', 'Artefact_Placeholder',
       'Artefact_Quantity', 'City_Bonus', 'Crafting Fame', 'Journal_Empty',
       'Journal_Full', 'Journal_EXP_Fill', 'Focus Cost', 'Sell_City',
       'Sell_Price', 'Sell_Price_Date', 'Main_Resource_City',
       'Main_Resource_Price', 'Main_Resource_Date', 'Secondary_Resource_City',
       'Secondary_Resource_Price', 'Secondary_Resource_Date', 'Artefact_Price',
       'Artefact_Date', 'Journal_Empty_Price', 'Journal_Full_Price',
       'Missing_Artefact', 'Missing_Second_Resource',
       'Secondary_Resource_Price_nn', 'Artefact_Price_nn', 'Tier_Enchant'],
      dtype='object')

In [14]:
BM_Craft

,index,Item_Placeholder,Tier,Enchant,Item_Name,Main_Resource,Main Resource Quantity,Secondary_Resource,Secondary_Resource_Quantity,Artefact_Placeholder,...,Secondary_Resource_Date,Artefact_Price,Artefact_Date,Journal_Empty_Price,Journal_Full_Price,Missing_Artefact,Missing_Second_Resource,Secondary_Resource_Price_nn,Artefact_Price_nn,Tier_Enchant
0,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-05-27T14:35:00,NaN,None,2515,2744,0,0,139,0,4.0
1,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-05-27T12:50:00,NaN,None,2515,2744,0,0,127,0,4.0
2,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-05-27T11:30:00,NaN,None,2515,2744,0,0,125,0,4.0
3,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-05-27T09:35:00,NaN,None,2515,2744,0,0,122,0,4.0
4,1732,T4_2H_ARCANESTAFF,4,0,Adept's Great Arcane Staff,T4_PLANKS,20,T4_METALBAR,12,0,...,2023-05-27T10:45:00,NaN,None,2515,2744,0,0,119,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118050,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,40264.0,2023-05-21T13:55:00,16059,99214,0,0,0,40264,8.3
118051,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,40264.0,2023-05-21T13:55:00,16059,99214,0,0,0,40264,8.3
118052,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,40264.0,2023-05-21T13:55:00,16059,99214,0,0,0,40264,8.3
118053,1699,T8_SHOES_PLATE_UNDEAD@3,8,3,Elder's Graveguard Boots,T8_METALBAR_LEVEL3@3,8,None,0,T8_ARTEFACT_SHOES_PLATE_UNDEAD,...,None,40264.0,2023-05-21T13:55:00,16059,99214,0,0,0,40264,8.3


In [15]:
# moving back to Pandas for final non-merge operations

BM_Craft['Secondary_Resource_Price'] = BM_Craft['Secondary_Resource_Price_nn']
BM_Craft = BM_Craft.drop('Secondary_Resource_Price_nn', axis=1)

BM_Craft['Artefact_Price'] = BM_Craft['Artefact_Price_nn']
BM_Craft = BM_Craft.drop('Artefact_Price_nn', axis=1)

BM_Craft['Resource_Costs'] = (BM_Craft['Main Resource Quantity']*BM_Craft['Main_Resource_Price'])+(BM_Craft['Secondary_Resource_Quantity']*BM_Craft['Secondary_Resource_Price'])
BM_Craft['Artefact_Costs'] = BM_Craft['Artefact_Quantity']*BM_Craft['Artefact_Price']
BM_Craft['Journal_Profit'] = ((BM_Craft['Journal_Full_Price']*0.975)-BM_Craft['Journal_Empty_Price'])*(BM_Craft['Crafting Fame']/BM_Craft['Journal_EXP_Fill'])

BM_Craft['Profit_City'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.752)+BM_Craft['Journal_Profit']
BM_Craft['Profit_City_Focus'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.502)+BM_Craft['Journal_Profit']
BM_Craft['Profit_City_BZ_Q6'] = (BM_Craft['Sell_Price']*0.945)-BM_Craft['Artefact_Costs']-(BM_Craft['Resource_Costs']*0.598)+BM_Craft['Journal_Profit']


BM_Craft = BM_Craft[['Item_Name','City_Bonus','Tier','Enchant','Tier_Enchant','Profit_City','Profit_City_Focus','Profit_City_BZ_Q6','Sell_City','Sell_Price','Sell_Price_Date','Main_Resource','Main_Resource_Price','Main_Resource_City','Main_Resource_Date','Secondary_Resource','Secondary_Resource_Price','Secondary_Resource_City','Secondary_Resource_Date','Artefact_Costs','Journal_Profit','Resource_Costs']]

In [16]:
BM_Craft.columns

Index(['Item_Name', 'City_Bonus', 'Tier', 'Enchant', 'Tier_Enchant',
       'Profit_City', 'Profit_City_Focus', 'Profit_City_BZ_Q6', 'Sell_City',
       'Sell_Price', 'Sell_Price_Date', 'Main_Resource', 'Main_Resource_Price',
       'Main_Resource_City', 'Main_Resource_Date', 'Secondary_Resource',
       'Secondary_Resource_Price', 'Secondary_Resource_City',
       'Secondary_Resource_Date', 'Artefact_Costs', 'Journal_Profit',
       'Resource_Costs'],
      dtype='object')

In [17]:
BM_Craft

,Item_Name,City_Bonus,Tier,Enchant,Tier_Enchant,Profit_City,Profit_City_Focus,Profit_City_BZ_Q6,Sell_City,Sell_Price,...,Main_Resource_Price,Main_Resource_City,Main_Resource_Date,Secondary_Resource,Secondary_Resource_Price,Secondary_Resource_City,Secondary_Resource_Date,Artefact_Costs,Journal_Profit,Resource_Costs
0,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,2030.164000,3142.164000,2715.156000,Black Market,5620,...,139,Bridgewatch,2023-05-27T14:40:00,T4_METALBAR,139,Bridgewatch,2023-05-27T14:35:00,0,64.160000,4448
1,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,2138.452000,3214.452000,2801.268000,Black Market,5620,...,139,Bridgewatch,2023-05-27T14:40:00,T4_METALBAR,127,Fort Sterling,2023-05-27T12:50:00,0,64.160000,4304
2,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,2156.500000,3226.500000,2815.620000,Black Market,5620,...,139,Bridgewatch,2023-05-27T14:40:00,T4_METALBAR,125,Lymhurst,2023-05-27T11:30:00,0,64.160000,4280
3,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,2183.572000,3244.572000,2837.148000,Black Market,5620,...,139,Bridgewatch,2023-05-27T14:40:00,T4_METALBAR,122,Martlock,2023-05-27T09:35:00,0,64.160000,4244
4,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,2210.644000,3262.644000,2858.676000,Black Market,5620,...,139,Bridgewatch,2023-05-27T14:40:00,T4_METALBAR,119,Thetford,2023-05-27T10:45:00,0,64.160000,4208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118050,Elder's Graveguard Boots,Martlock,8,3,8.3,-112006.165433,345791.834567,169997.402567,Fort Sterling,1299994,...,228899,Bridgewatch,2023-05-24T07:25:00,None,0,None,None,40264,76819.888567,1831192
118051,Elder's Graveguard Boots,Martlock,8,3,8.3,-105990.165433,349807.834567,174781.402567,Fort Sterling,1299994,...,227899,Fort Sterling,2023-05-27T13:50:00,None,0,None,None,40264,76819.888567,1823192
118052,Elder's Graveguard Boots,Martlock,8,3,8.3,-94565.781433,357434.218567,183866.218567,Fort Sterling,1299994,...,226000,Lymhurst,2023-05-25T18:10:00,None,0,None,None,40264,76819.888567,1808000
118053,Elder's Graveguard Boots,Martlock,8,3,8.3,-177712.917433,301929.082567,117746.554567,Fort Sterling,1299994,...,239821,Martlock,2023-05-27T14:55:00,None,0,None,None,40264,76819.888567,1918568


In [18]:
BM_Craft.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\report.csv", index=False)